In [1]:
import itertools
import sys
import math

from sklearn.model_selection import train_test_split, KFold


data_path = '/Users/tyamgin/Projects/mlbootcamp/vkcup2021_2/data/'

In [37]:
%run /Users/tyamgin/Projects/mlbootcamp/vkcup2021_2/lib.py

In [30]:
train = Data()
train.read(data_path, 'train')

In [25]:
def expand_grid(dictionary):
   return pd.DataFrame([row for row in itertools.product(*dictionary.values())], 
                       columns=dictionary.keys())

In [26]:
def rmse(x, y):
    n = len(x)
    return math.sqrt(1.0 * sum((x - y)**2) / n)

In [31]:
def cross_validation(model, data, n_folds=5, n_iters=5, seed=2707, verbose=1):
    scores = []
    user_ids = data.edu['uid'].unique()
    
    for iter in range(n_iters):
        kf = KFold(n_splits=n_folds, random_state=seed+iter, shuffle=True)
        print(kf)
        fold = 0
        for train_index, test_index in kf.split(user_ids):
            fold += 1
            if verbose >= 2:
                print('Prepare data: %s' % datetime.datetime.now())
                
            data_train = data.get(user_ids[train_index])
            data_test = data.get(user_ids[test_index])

            if verbose >= 2:
                print('Fit: %s' % datetime.datetime.now())
            model.fit(data_train)
            if verbose >= 2:
                print('Predict: %s' % datetime.datetime.now())

            pred = model.predict(data_test)
            if verbose >= 2:
                print('Auc: %s' % datetime.datetime.now())
            score = rmse(data_test.edu['age'].values, pred['res'].values)
            scores.append(score)

            #if verbose >= 1:
            print('%2d -%2d : %1.4f, mean=%1.4f' % (iter, fold, score, np.mean(scores)))
    return np.mean(scores)

In [33]:
class SimpleModel(MyModel):
    def predict(self, X):
        edu = X.edu.copy()
        edu['res'] = 34
        dct = {
            'school_education': 17,  # 4.903859550418915
            'graduation_1': 25,  # 5.590608371258384
            'graduation_2': 24,  # 4.912185675064848
            'graduation_3': 24,  # 4.905354217587146
            'graduation_4': 25,  # 4.009603855360661
            'graduation_5': 24,  # 5.55578449198568
            'graduation_6': 25,  # 5.225676253624243
            'graduation_7': 25,  # 7.671183665089072
        }
        for grad, shift in dct.items():
            grad_idx = ~edu[grad].isna()
            edu.loc[grad_idx, 'res'] = 2021 - edu.loc[grad_idx, grad] + shift

        return edu[['uid', 'res']].set_index('uid')

model = SimpleModel()
cross_validation(model, train, n_folds=5, n_iters=2, verbose=1)

KFold(n_splits=5, random_state=2707, shuffle=True)
 0 - 1 : 11.5868, mean=11.5868
 0 - 2 : 11.6415, mean=11.6141
 0 - 3 : 11.3214, mean=11.5166
 0 - 4 : 11.2377, mean=11.4469
 0 - 5 : 11.5750, mean=11.4725
KFold(n_splits=5, random_state=2708, shuffle=True)
 1 - 1 : 11.4767, mean=11.4732
 1 - 2 : 11.3931, mean=11.4618
 1 - 3 : 11.3912, mean=11.4529
 1 - 4 : 11.5425, mean=11.4629
 1 - 5 : 11.5635, mean=11.4730


11.47295633693525

In [38]:
grid = expand_grid({
    'boosting_type': ['gbdt'],
    'min_data_in_leaf': [25],
    'lambda_l2': [0.0],
    'num_leaves': [10],
    'learning_rate': [0.03],
    'feature_fraction': [1],
    'bagging_fraction': [1],
    'bagging_freq': [5],
    'num_boost_round': [200],
})
min_rmse = 1e10
best_params = None
for i, r in grid.iterrows():
    model = LgbModel({
        'boosting_type': r.boosting_type,
        'min_data_in_leaf': r.min_data_in_leaf,
        'lambda_l2': r.lambda_l2,
        'num_leaves': r.num_leaves,
        'learning_rate': r.learning_rate,
        'feature_fraction': r.feature_fraction,
        'bagging_fraction': r.bagging_fraction,
        'bagging_freq': r.bagging_freq,
        'num_boost_round': r.num_boost_round,
    })
    rr = cross_validation(model, train, n_folds=5, n_iters=1, verbose=0)
    if rr < min_rmse:
        min_rmse = rr
        best_params = r.copy()
    print(r)
    print(rr)
        

KFold(n_splits=5, random_state=2707, shuffle=True)
Starting train: 2021-07-26 17:44:31.639588
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001019 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 519
[LightGBM] [Info] Number of data points in the train set: 23575, number of used features: 10
[LightGBM] [Info] Start training from score 35.017179
 0 - 1 : 11.4868, mean=11.4868
Starting train: 2021-07-26 17:44:31.922411
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000745 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 521
[LightGBM] [Info] Number of data points in the train set: 23575, number of used features: 10
[LightGBM] [Info] Start training from score 34.965302
 0 - 2 : 11.56

In [254]:
print(min_rmse)
print(best_params)

11.390829067278895
boosting_type       gbdt
min_data_in_leaf      25
lambda_l2            0.0
num_leaves            10
learning_rate       0.03
feature_fraction       1
bagging_fraction       1
bagging_freq           5
num_boost_round      200
Name: 0, dtype: object


In [45]:
prod_model = LgbModel({
    'boosting_type': 'gbdt',
    'min_data_in_leaf': 25,
    'lambda_l2': 0.0,
    'num_leaves': 10,
    'learning_rate': 0.03,
    'feature_fraction': 1,
    'bagging_fraction': 1,
    'bagging_freq': 5,
    'num_boost_round': 200,
})
prod_model.fit(train)
prod_model.model.save_model('/Users/tyamgin/Projects/mlbootcamp/vkcup2021_2/models/lgb.txt')
prod_model.predict(train)

Starting train: 2021-07-26 18:03:32.873157
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000867 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 537
[LightGBM] [Info] Number of data points in the train set: 29469, number of used features: 10
[LightGBM] [Info] Start training from score 35.029149


,uid,res
0,356,32.535072
1,26356,37.879789
2,97,34.240528
3,319,31.294645
4,30288,31.471455
...,...,...
29464,33331,50.091281
29465,119788,34.531147
29466,97581,36.624310
29467,104038,67.178940


In [150]:
train_edu

,school_education,graduation_1,graduation_2,graduation_3,graduation_4,graduation_5,graduation_6,graduation_7,age,registered_year
uid,,,,,,,,,,
356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48,2017
26356,NaN,NaN,NaN,NaN,NaN,2006.0,NaN,NaN,49,2008
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35,2011
319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37,2020
30288,2008.0,NaN,NaN,NaN,NaN,2013.0,NaN,NaN,30,2008
...,...,...,...,...,...,...,...,...,...,...
33331,1987.0,NaN,NaN,NaN,NaN,1991.0,NaN,NaN,51,2008
119788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,2013
97581,2002.0,NaN,NaN,NaN,NaN,2007.0,NaN,NaN,36,2010


In [103]:
for grad in ('school_education', 'graduation_1', 'graduation_2',  'graduation_3', 'graduation_4',
             'graduation_5', 'graduation_6', 'graduation_7'):
    gg = train_edu[~train_edu[grad].isna()].set_index('uid').join(train.set_index('uid'), how='left')
    best_age = -1
    min_rmse = 1e10
    for age in range(10,  80):
        r = rmse(2021 - gg[grad] + age, gg.age)
        if r < min_rmse:
            best_age = age
            min_rmse = r
    print(f"'{grad}': {best_age},  # {min_rmse}")
        

'school_education': 17,  # 4.903859550418915
'graduation_1': 25,  # 5.590608371258384
'graduation_2': 24,  # 4.912185675064848
'graduation_3': 24,  # 4.905354217587146
'graduation_4': 25,  # 4.009603855360661
'graduation_5': 24,  # 5.55578449198568
'graduation_6': 25,  # 5.225676253624243
'graduation_7': 25,  # 7.671183665089072


In [46]:
rmse(2021 - gg.school_education + 17, gg.age)

4.903859550418915

In [121]:
def solve(edu):
    edu['res'] = 34
    dct = {
        'school_education': 17,  # 4.903859550418915
        'graduation_1': 25,  # 5.590608371258384
        'graduation_2': 24,  # 4.912185675064848
        'graduation_3': 24,  # 4.905354217587146
        'graduation_4': 25,  # 4.009603855360661
        'graduation_5': 24,  # 5.55578449198568
        'graduation_6': 25,  # 5.225676253624243
        'graduation_7': 25,  # 7.671183665089072
    }
    for grad, shift in dct.items():
        grad_idx = ~edu[grad].isna()
        edu.loc[grad_idx, 'res'] = 2021 - edu.loc[grad_idx, grad] + shift

    return edu[['uid', 'res']].set_index('uid')
    
res = solve(train_edu)
res

,res
uid,
356,34.0
26356,39.0
97,34.0
319,34.0
30288,32.0
...,...
33331,54.0
119788,34.0
97581,38.0


In [122]:
res_joined = res.join(train.set_index('uid'), how='left')
rmse(res_joined.res, res_joined.age)

11.47363369861934

In [ ]:
# 13.12131431455484 - 35
# 11.585301610692005 - 35 + school
# 11.512968371985176 - 35 + all
# 11.47363369861934 - 34 + all